In [1]:
# Import required libraries
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import os


In [2]:
# Set parameters
INIT_LR = 1e-4
EPOCHS = 20
BS = 32
IMG_SIZE = 224


In [16]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode="nearest",
    validation_split=0.2
)


In [17]:
# Update the path to match your local directory
DATASET_DIR = "/Users/sakshikiranjadhav/Desktop/dataset_maskdetection"

train_gen = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BS,
    class_mode="categorical",
    subset="training"
)

val_gen = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BS,
    class_mode="categorical",
    subset="validation"
)

Found 6043 images belonging to 2 classes.
Found 1510 images belonging to 2 classes.


In [18]:
# Load the MobileNetV2 model without top layers
baseModel = MobileNetV2(weights="imagenet", include_top=False,
                        input_tensor=Input(shape=(IMG_SIZE, IMG_SIZE, 3)))

/var/folders/5w/k3xs6p855z589rp7n4t1yhyr0000gn/T/ipykernel_27444/2412811890.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  baseModel = MobileNetV2(weights="imagenet", include_top=False,


In [19]:
# Freeze base layers
for layer in baseModel.layers:
    layer.trainable = False
    
from tensorflow.keras import regularizers

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten()(headModel)
headModel = Dense(128, activation="relu", kernel_regularizer=regularizers.l2(0.001))(headModel)
headModel = Dropout(0.6)(headModel)
headModel = Dense(2, activation="softmax")(headModel)


In [20]:
# Final model
model = Model(inputs=baseModel.input, outputs=headModel)

In [21]:
# Compile model
model.compile(loss="categorical_crossentropy",
              optimizer=Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS),
              metrics=["accuracy"])


In [22]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ModelCheckpoint("best_model.h5", monitor="val_accuracy", save_best_only=True)
]

history = model.fit(
    train_gen,
    steps_per_epoch=train_gen.samples // BS,
    validation_data=val_gen,
    validation_steps=val_gen.samples // BS,
    epochs=EPOCHS,
    callbacks=callbacks
)

Epoch 1/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 593ms/step - accuracy: 0.7336 - loss: 0.7887

188/188 ━━━━━━━━━━━━━━━━━━━━ 155s 812ms/step - accuracy: 0.7343 - loss: 0.7875 - val_accuracy: 0.9561 - val_loss: 0.3462
Epoch 2/20
  1/188 ━━━━━━━━━━━━━━━━━━━━ 56s 304ms/step - accuracy: 0.9688 - loss: 0.3528

188/188 ━━━━━━━━━━━━━━━━━━━━ 39s 210ms/step - accuracy: 0.9688 - loss: 0.3528 - val_accuracy: 0.9641 - val_loss: 0.3337
Epoch 3/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 155s 825ms/step - accuracy: 0.9430 - loss: 0.3579 - val_accuracy: 0.9621 - val_loss: 0.3155
Epoch 4/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 33s 174ms/step - accuracy: 0.9062 - loss: 0.3834 - val_accuracy: 0.9634 - val_loss: 0.3114
Epoch 5/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 650ms/step - accuracy: 0.9647 - loss: 0.3104

188/188 ━━━━━━━━━━━━━━━━━━━━ 152s 808ms/step - accuracy: 0.9647 - loss: 0.3104 - val_accuracy: 0.9701 - val_loss: 0.2950
Epoch 6/20
  1/188 ━━━━━━━━━━━━━━━━━━━━ 1:00 326ms/step - accuracy: 0.9375 - loss: 0.3050

188/188 ━━━━━━━━━━━━━━━━━━━━ 33s 177ms/step - accuracy: 0.9375 - loss: 0.3050 - val_accuracy: 0.9747 - val_loss: 0.2852
Epoch 7/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 146s 776ms/step - accuracy: 0.9647 - loss: 0.2918 - val_accuracy: 0.9707 - val_loss: 0.2843
Epoch 8/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 29s 153ms/step - accuracy: 1.0000 - loss: 0.2349 - val_accuracy: 0.9707 - val_loss: 0.2842
Epoch 9/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 150s 798ms/step - accuracy: 0.9751 - loss: 0.2669 - val_accuracy: 0.9721 - val_loss: 0.2795
Epoch 10/20
  1/188 ━━━━━━━━━━━━━━━━━━━━ 1:26 463ms/step - accuracy: 0.9688 - loss: 0.2739

188/188 ━━━━━━━━━━━━━━━━━━━━ 28s 145ms/step - accuracy: 0.9688 - loss: 0.2739 - val_accuracy: 0.9754 - val_loss: 0.2612
Epoch 11/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 150s 800ms/step - accuracy: 0.9755 - loss: 0.2599 - val_accuracy: 0.9747 - val_loss: 0.2537
Epoch 12/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 33s 176ms/step - accuracy: 0.9688 - loss: 0.2865 - val_accuracy: 0.9701 - val_loss: 0.2657
Epoch 13/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 636ms/step - accuracy: 0.9709 - loss: 0.2496

188/188 ━━━━━━━━━━━━━━━━━━━━ 150s 797ms/step - accuracy: 0.9709 - loss: 0.2496 - val_accuracy: 0.9801 - val_loss: 0.2444
Epoch 14/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 29s 156ms/step - accuracy: 0.9688 - loss: 0.2335 - val_accuracy: 0.9781 - val_loss: 0.2374
Epoch 15/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 145s 769ms/step - accuracy: 0.9766 - loss: 0.2464 - val_accuracy: 0.9707 - val_loss: 0.2488
Epoch 16/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 30s 159ms/step - accuracy: 1.0000 - loss: 0.1827 - val_accuracy: 0.9761 - val_loss: 0.2369
Epoch 17/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 147s 783ms/step - accuracy: 0.9728 - loss: 0.2364 - val_accuracy: 0.9761 - val_loss: 0.2376
Epoch 18/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 28s 145ms/step - accuracy: 1.0000 - loss: 0.2048 - val_accuracy: 0.9761 - val_loss: 0.2415
Epoch 19/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 152s 810ms/step - accuracy: 0.9785 - loss: 0.2239 - val_accuracy: 0.9774 - val_loss: 0.2243
Epoch 20/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 33s 176ms/step - accuracy: 1.0000 - loss: 0

In [ ]:
# Save model
model.save("mask_detector_mobilenetv2.h5")